In [1]:
%matplotlib inline

from cycler import cycler

import matplotlib as mpl
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.colors
from matplotlib.collections import LineCollection
from matplotlib.colors import BoundaryNorm


import seaborn as sns
sns.set()
#sns.set_style("whitegrid")
#current_palette = sns.color_palette("Blues_r")
#plt.set_palette(sns.color_palette("Blues_r"))
#plt.style.use("seaborn-darkgrid")


import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import numpy.ma as ma

from scipy import integrate
from scipy import interpolate
from scipy.optimize import curve_fit

import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

#permutations
import itertools

from operator import itemgetter
from operator import attrgetter

import math

Аналитическое решение квадратного уравнения

In [2]:
# ax**2 + bx + c = 0
def quadratic_equation_root(a, b, c):
    d = (b**2) - (4*a*c)
    return (-b + d**0.5)/(2*a)

Преобразование единиц измерения напора, расхода и мощности

In [3]:
def convert_unit(kind, value, unit_in, unit_out):
    unit = {
        'head': {'m':1.0, 'kgf_cm2':10.0, 'bar':10.197, 'kPa':0.10197, 'atm':10.33, 'psi':0.70307, 'mmHg': 0.013595},
        'flow': {'cbmh':1.0, 'cbms':3600.0, 'lpm':0.06, 'lps':3.6},
        'power': {'kW':1.0, 'W':0.001}
    }
    return value * unit[kind][unit_in] / unit[kind][unit_out]

Набор цветов для графиков

In [4]:
def categorical_cmap(nc, nsc, cmap="tab10", continuous=False):
    if nc > plt.get_cmap(cmap).N:
        raise ValueError("Too many categories for colormap")
    if continuous:
        ccolors = plt.get_cmap(cmap)(np.linspace(0,1,nc))
    else:
        ccolors = plt.get_cmap(cmap)(np.arange(nc, dtype=int))
    cols = np.zeros((nc*nsc, 3))
    for i, c in enumerate(ccolors):
        chsv = matplotlib.colors.rgb_to_hsv(c[:3])
        arhsv = np.tile(chsv,nsc).reshape(nsc,3)
        arhsv[:,1] = np.linspace(chsv[1],0.25,nsc)
        arhsv[:,2] = np.linspace(chsv[2],1,nsc)
        rgb = matplotlib.colors.hsv_to_rgb(arhsv)
        cols[i*nsc:(i+1)*nsc,:] = rgb
    cmap = matplotlib.colors.ListedColormap(cols)
    return cmap

Поиск индекса элемента в массиве по значению

In [5]:
def find_index(array, value):
    array = np.asarray(array)
    return (np.abs(array - value)).argmin()

Поиск ближайшего значения из массива

In [1]:
def find_quant(array, value):
    idx = find_index(array, value)
    return array[idx]

Формат парсера времени

In [2]:
dateparser = lambda x: pd.datetime.strptime(x, "%d.%m.%Y %H:%M:%S")

Интегрирование TimeSeries

In [ ]:
def integrate_method(self, how='trapz', unit='h'):
    '''Numerically integrate the time series.

    @param how: the method to use (trapz by default)
    @return 

    Available methods:
     * trapz - trapezoidal
     * cumtrapz - cumulative trapezoidal
     * simps - Simpson's rule
     * romb - Romberger's rule

    See http://docs.scipy.org/doc/scipy/reference/integrate.html for the method details.
    or the source code
    https://github.com/scipy/scipy/blob/master/scipy/integrate/quadrature.py
    '''
    available_rules = set(['trapz', 'cumtrapz', 'simps', 'romb'])
    if how in available_rules:
        rule = integrate.__getattribute__(how)
    else:
        print('Unsupported integration rule: %s' % (how))
        print('Expecting one of these sample-based integration rules: %s' % (str(list(available_rules))))
        raise AttributeError
    
    result = rule(self.values, self.index.astype(np.int64) / 10**9)
    #result = rule(self.values)
    return result
pd.Series.integrate = integrate_method

### Насос

Для каждого насоса должны задаваться рабочие характеристики - напора и мощности на валу.
Также должны указываться минимальный и максимальный допустимый расход на номинальной характеристике.
Для расчета КПД необходимо указать два значения КПД на кривой насоса, одно из которых должно быть максимальным значением.

Расход, напор и мощность на разных скоростях вращения  рассчитываются через законы подобия.

$\frac{Q_2}{Q_1} = \frac{n_2}{n_1} = \omega$ , $\frac{H_2}{H_1} = \left(\frac{n_2}{n_1}\right)^2 = \omega^2$, $\frac{P_2}{P_1} = \left(\frac{n_2}{n_1}\right)^3 = \omega^3$.

Кривая напора насоса аппроксимируется кривой второго порядка:

$H(Q) = H_0 - C_1 \cdot Q - С_2 \cdot Q^2$

Для расчета КПД насоса в любой точке также используется парабола. Если известно только значение максимального КПД $(Q_{BEP}, \eta_{BEP})$, кривая задается формулой

$\eta(Q) = a \cdot Q^2 + b \cdot Q$,

коэффициенты которой находятся из следующей системы уравнений:

$\left\{
	\begin{aligned}
		&a \cdot Q^2_{BEP} + b \cdot Q_{BEP} - \eta_{BEP} = 0 \\
		&2a \cdot Q_{BEP} + b = 0
	\end{aligned}
\right.
$

Если известно две точки $(Q_{BEP}, \eta_{BEP})$ и $(Q_2, \eta_2)$, то кривая задается следующим соотношением

$\eta(Q) = 
	\begin{cases}
		a \cdot Q^2 + b \cdot Q, &\text{если }  Q < Q_{BEP} \\
		a_2 \cdot Q^2 + b_2 \cdot Q + c_2, &\text{если }  Q \ge Q_{BEP}
	\end{cases}
$

где коэффициенты $a$ и $b$ находятся из системы уравнений выше, а остальные из следующей системы 

$
	\left\{
		\begin{aligned}
			&a_2 \cdot Q^2_{BEP} + b_2 \cdot Q_{BEP} + c_2 - \eta_{BEP} = 0 \\
			&2a_2 \cdot Q_{BEP} + b_2 = 0 \\
			&a_2 \cdot Q^2_2 + b_2 \cdot Q_2 + c_2 - \eta_2 = 0
		\end{aligned}
	\right.
$

Зависимость гидравлического КПД насоса от скорости вращения рассчитывается, используя

$\eta_P = \eta_2 = 1 - (1 - \eta_1) \cdot \left(\frac{n_1}{n_2}\right)^{0,1} = 1 - (1 - \eta_1) \cdot \left(\frac{1}{\omega}\right)^{0,1}$.


Гидравлическая мощность:

$P_H = \rho gQH$,

и мощность на валу:

$P_S = \frac{P_H}{\eta_P}$.

Эта мощность на валу насоса является активной мощностью ны выходе двигателя.

In [6]:
# описание насоса
class Pump:

    def __init__(self, df):
        self.flow_rated_values = df.Q
        self.head_rated_values = df.H
        self.flow_rated_min = df.Q_min[0]
        self.flow_rated_max = df.Q_max[0]
        self.flow_rated_bep = df.Q_bep[0]
        self.flow_rated_bep_2 = df.Q_bep2[0]
        self.eff_rated_bep = df.eta_bep[0]
        self.eff_rated_bep_2 = df.eta_bep2[0]
        self.speed = 1.0
        self.head_equation_solve()
        self.eff_equation_solve()
        self.speed_allow = 0.25
        self.set_ranges()
        #self.model = df.Model[0]

    def set_speed(self, speed):
        self.speed = speed
        self.set_ranges()

    def set_ranges(self):
        self.flow_min = self.affinity_law('flow', self.flow_rated_min, self.speed)
        self.flow_max = self.affinity_law('flow', self.flow_rated_max, self.speed)
        self.head_rated_range()
        self.head_min = self.affinity_law('head', self.head_rated_min, self.speed)
        self.head_max = self.affinity_law('head', self.head_rated_max, self.speed)
        self.head_allow_range()

    def head_rated_range(self):
        self.head_rated_min = self.head(self.flow_rated_max, 1.0)
        self.flow_head_max = -0.5 * (self.head_c[1] / self.head_c[2])
        if (self.flow_head_max < self.flow_rated_min) or (self.flow_head_max < 0.0):
            self.head_rated_max = self.head(self.flow_rated_min, 1.0)
        else:
            self.head_rated_max = self.head(self.flow_head_max, 1.0)

    def head_allow_range(self):
        self.head_allow_max = self.head_rated_max
        flow_allow_max = self.affinity_law('flow', self.flow_rated_max, self.speed_allow)
        self.head_allow_min = self.head(flow_allow_max, self.speed_allow)
    
    def head_equation(self, flow, c0, c1, c2):
        return c0 - c1*flow - c2*flow**2

    def head_equation_solve(self):
        c = [0, 0, 0]
        self.head_c, cov = curve_fit(self.head_equation, self.flow_rated_values, self.head_rated_values, c)

    def head(self, flow, speed):
        return self.head_c[0] * speed**2 - self.head_c[1]*flow*speed - self.head_c[2] * flow**2

    def flow(self, head, speed):
        self.set_speed(speed)
        root = quadratic_equation_root(self.head_c[2], self.head_c[1]*speed, head - (self.head_c[0] * speed**2))       
        root[head > self.head_max] = 0
        root[head < self.head_min] = 0
        return root
    
    def flow_eff_max(self, speed):
        return self.eff_max()*speed

    def eff_max(self):
        return -self.eff_b/2.0/self.eff_a
    
    def eff_equation(self, flow, a, b, c):
        return a * flow**2 + b*flow + c

    def eff_equation_solve1d(self):
        A = np.array([[self.flow_rated_bep**2, self.flow_rated_bep], [2*self.flow_rated_bep, 1]])
        b = np.array([self.eff_rated_bep, 0])
        z = np.linalg.solve(A, b)
        self.eff_a = z[0]
        self.eff_b = z[1]

    def eff_equation_solve2d(self):
        A = np.array([[self.flow_rated_bep**2, self.flow_rated_bep, 1], [2*self.flow_rated_bep, 1, 0],
                      [self.flow_rated_bep_2**2, self.flow_rated_bep_2, 1]])
        b = np.array([self.eff_rated_bep, 0, self.eff_rated_bep_2])
        z = np.linalg.solve(A, b)
        self.eff_a2 = z[0]
        self.eff_b2 = z[1]
        self.eff_c2 = z[2]

    def eff_equation_solve(self):
        self.eff_equation_solve1d()
        if not math.isnan(self.flow_rated_bep_2):
            self.eff_equation_solve2d()

    def eff(self, flow, speed):
        flow_by_speed = flow / speed
        if math.isnan(self.flow_rated_bep_2):
            return self.eff_equation(flow_by_speed, self.eff_a, self.eff_b, 0.0)
        else:
            flow1 = flow_by_speed[flow_by_speed <= self.flow_rated_bep]
            flow2 = flow_by_speed[flow_by_speed > self.flow_rated_bep]
            eff1 = self.eff_equation(flow1, self.eff_a, self.eff_b, 0.0)
            eff2 = self.eff_equation(flow2, self.eff_a2, self.eff_b2, self.eff_c2)
            return np.concatenate((eff1, eff2))

    def power_hydro(self, flow, speed):
        head = self.head(flow, speed)
        return 998.0 * 9.81 * head * flow / 3600.0 / 1000.0

    def power_shaft(self, flow, speed):
        return self.power_hydro(flow, speed) / self.eff(flow, speed) * 100.0

    def affinity_law(self, kind, value, speed):
        kind_to_multiplier = {'flow': speed, 'head': speed**2, 'power': speed**3}
        return value * kind_to_multiplier[kind]

    def plot_work_curves(self, flow_step=0.1, speed_count=6, flow_xlim=None):
        cmap = categorical_cmap(1, speed_count+1, cmap="Blues_r")
        fig, axes = plt.subplots(nrows=3, ncols=1,
                                 sharex=True, sharey=False,
                                 figsize=(7, 10),
                                 constrained_layout=True,
                                 gridspec_kw={'height_ratios':[2,1,1]})
        fig.set_constrained_layout_pads(w_pad=4./72., h_pad=4./72., hspace=0.0, wspace=0.)
        idx = 0
        speed = np.linspace(1.0, self.speed_allow, speed_count)
        for s in speed:
            self.set_speed(s)
            q = np.arange(self.flow_min, self.flow_max, flow_step)
            q_max = self.flow_eff_max(s)
            axes[0].plot(q, self.head(q, s), label=s, color=cmap(idx))
            axes[0].scatter(q_max, self.head(q_max, s), color=cmap(idx))
            axes[1].plot(q, self.power_hydro(q, s), color=cmap(idx))
            axes[1].scatter(q_max, self.power_hydro(q_max, s), color=cmap(idx))
            axes[2].plot(q, self.eff(q, s), color=cmap(idx))
            axes[2].scatter(q_max, self.eff(q_max, s), color=cmap(idx))
            idx += 1
        fig.suptitle('Рабочие характеристики насоса')
        axes[0].legend(title='Скорость')#, bbox_to_anchor=(0.96, 0.94), loc='upper right', borderaxespad=0.0)
        axes[0].set_ylabel('$H,\, м$')
        axes[0].set_ylim(0, None)
        axes[1].set_ylabel('$P_H,\, кВт$')
        axes[1].set_ylim(0, None)
        axes[2].set_ylabel('$\eta_H,\, \%$')
        axes[2].set_ylim(0, None)
        plt.xlabel('$Q,\, м^3/ч$')
        plt.axis([0, flow_xlim, None, None])
        plt.show()

### Двигатель

Для двигателя указываются номинальная мощность $P_{NOM}$ и значения КПД при нагрузках $100\,\%$ и $75\,\%$ -- $\eta_{M,100}$ и $\eta_{M,75}$, соответственно.

Нагрузка двигателя:

$L = \frac{P_S}{\frac{P_{NOM}}{\eta_{M,100}}}$


Согласно стандарту IEC 60034-31 можно рассчитать КПД двигателя, при любом значении нагрузки по формуле:

$	\begin{aligned}
		&\nu_L = \frac{ \left(\frac{1}{\eta_{M,100}}-1\right) - 0,75 \left(\frac{1}{\eta_{M,75}}-1\right)}{0,4375}, \\
		&\nu_0 = \left(\frac{1}{\eta_{M,100}}-1\right)-\nu_L, \\
		&\eta_M = \frac{1}{1+\frac{\nu_0}{L}+\nu_L \times L}.
	\end{aligned}
$

Отсюда, потребляемая мощность двигателя:

$P_M = \frac{P_S}{\eta_M}$,

In [28]:
# описание двигателя
class Motor:

    def __init__(self, df):
        self.rpm = df.rpm[0]
        self.eff_100 = df.eta_100[0]
        self.eff_75 = df.eta_75[0]
        self.power_rated = df.P_m[0]

    def load(self, power_shaft):
        return power_shaft / (self.power_rated / self.eff_100 * 100.0)

    def eff(self, power_shaft):
        v1 = ((1/self.eff_100*100.0 - 1) - 0.75 * (1/self.eff_75*100.0 - 1)) / 0.4375
        v0 = (1/self.eff_100*100.0 - 1) - v1
        load = self.load(power_shaft)
        return 1 / (1 + v0/load + v1*load) * 100.0

    def power_electric(self, power_shaft):
        return power_shaft / self.eff(power_shaft) * 100.0

    def power_shaft(self, load):
        return load * self.power_rated / self.eff_100

    def plot_eff_curve(self):
        load = np.arange(1.0, 100.0, 0.1)
        power_shaft = self.power_shaft(load)
        fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 5))
        fig.set_constrained_layout_pads(w_pad=4./72., h_pad=4./72., hspace=0.0, wspace=0.)
        fig.suptitle('Характеристика КПД двигателя')
        axes.plot(load, self.eff(power_shaft))
        axes.set_ylabel('$\eta_M,\, \%$')
        axes.set_xlabel('Нагрузка, %')
        plt.axis([0, 100.0, 0, None])
        plt.show()

### Частотный преобразователь

Для современных частотных преобразователей можно считать КПД на всем диапазоне нагрузок постоянным. Принимаем, что $\eta_{VFD} = 97\%$, а потребляемая мощность электрической сети:

$P_E = \frac{P_M}{\eta_{VFD}}$

In [29]:
# описание частотного преобразователя
class VariableFrequencyDrive:

    def __init__(self):
        self.eff = 97

    def power_electric(self, power_supply):
        return power_supply * self.eff / 100.0
    
    def power_supply(self, power_electric):
        return power_electric / self.eff * 100.0

### Насосный агрегат
Общий КПД насосного агрегата складывается из КПД составляющих частей - насоса, двигателя и частотного преобразователя:

$\eta_{TOT} = \frac{P_H}{P_E} = \eta_P \cdot \eta_M \cdot \eta_{VFD}$.

In [1]:
class PumpUnit:

    def __init__(self, file_name):
        df = pd.read_csv(file_name, sep=';', decimal=',', index_col=False)
        self.pump = Pump(df)
        self.motor = Motor(df)
        self.vfd = VariableFrequencyDrive()
        self.hist_work = pd.Series([])
        self.hist_power_supply = pd.Series([])
        self.hist_power_electric = pd.Series([])
        self.hist_power_hydro = pd.Series([])
        self.hist_eff_total = pd.Series([])
        self.hist_freq = pd.Series([])
        self.hist_head_inflow = pd.Series([])
        self.hist_head_outflow = pd.Series([])
        self.hist_head = pd.Series([])        
        self.hist_flow = pd.Series([])

    def kind_historic(self, kind, file_name, link, convert):
        if kind == 'work':
            self.hs_work = HistoricSeries(file_name, link, discrete=True)
            self.hist_work = self.hs_work.adapt
        elif kind == 'power_electric':
            self.hs_power_electric = HistoricSeries(file_name, link)
            self.hist_power_electric = self.hs_power_electric.adapt
        elif kind == 'power_supply':
            self.hs_power_supply = HistoricSeries(file_name, link)
            self.hist_power_supply = self.hs_power_supply.adapt
        elif kind == 'frequency':
            self.hs_freq = HistoricSeries(file_name, link)
            self.hist_freq = self.hs_freq.adapt
        elif kind == 'head_inflow':
            self.hs_head_inflow = HistoricSeries(file_name, link, convert)
            self.hist_head_inflow = self.hs_head_inflow.adapt
        elif kind == 'head_outflow':
            self.hs_head_outflow = HistoricSeries(file_name, link, convert)
            self.hist_head_outflow = self.hs_head_outflow.adapt

    def eff(self, flow, speed):
        self.eff_pump = self.pump.eff(flow, speed)
        power_shaft = self.pump.power_shaft(flow, speed)
        self.eff_motor = self.motor.eff(power_shaft)
        self.eff_vfd = self.vfd.eff
        self.eff_total = self.eff_pump * self.eff_motor * self.eff_vfd / 10000.0
        return self.eff_total
    
    def power(self, flow, speed):
        self.power_hydro = self.pump.power_hydro(flow, speed)
        self.power_shaft = self.pump.power_shaft(flow, speed)
        self.power_electric = self.motor.power_electric(self.power_shaft)
        self.power_supply = self.vfd.power_supply(self.power_electric)
    
    def set_speed(self, speed):
        #self.motor.set_speed(speed)
        self.pump.set_speed(speed)

    def flow_idx_allow(self, head_points):
        self.flow_idx_min = []
        self.flow_idx_max = []
        for head_idx in range(len(head_points)):
            idx = np.argwhere(self.eff_matrix[head_idx] > 0.0)
            if not idx.size > 0:
                self.flow_idx_min.append(-1)
                self.flow_idx_max.append(-1)
            else:
                self.flow_idx_min.append(idx[0])
                self.flow_idx_max.append(idx[-1])

    def calc_matrices(self, flow_points, flow_num, head_points, head_num, speed_points):
        self.speed_matrix = np.zeros(head_num * flow_num)
        self.speed_matrix = self.speed_matrix.reshape((head_num, flow_num))
        self.eff_matrix = np.zeros(head_num*flow_num)
        self.eff_matrix = self.eff_matrix.reshape((head_num, flow_num))
        self.eff_opt = np.zeros(flow_num)
        self.speed_opt = np.zeros(flow_num)
        flow_idx = 0
        for flow in flow_points:
            speed_arr = []
            head_arr = []
            eff_arr = []
            for s in speed_points:
                self.set_speed(s)
                if self.pump.flow_min <= flow <= self.pump.flow_max:
                    eff_arr.append(self.eff(flow, s))
                    head_arr.append(self.pump.head(flow, s))
                    speed_arr.append(s)
            if not eff_arr:
                flow_idx += 1
                continue
            for idx, head in enumerate(head_arr):
                head_idx = find_index(head_points, head)
                self.eff_matrix[head_idx][flow_idx] = eff_arr[idx]
                self.speed_matrix[head_idx][flow_idx] = speed_arr[idx]
            self.eff_opt[flow_idx] = max(eff_arr)
            self.speed_opt[flow_idx] = speed_arr[np.argmax(eff_arr)]
            flow_idx += 1
        self.flow_idx_allow(head_points)
 
    def plot_work_curves(self, flow_step=0.1, speed_count=6, flow_xlim=None):
        cmap_count = speed_count + 2
        cmap = categorical_cmap(3, cmap_count, cmap="jet")
        fig, axes = plt.subplots(nrows=2, ncols=1,
                                 sharex=True, sharey=False,
                                 figsize=(7, 10),
                                 constrained_layout=True,
                                 gridspec_kw={'height_ratios':[1,1]})
        fig.set_constrained_layout_pads(w_pad=4./72., h_pad=4./72., hspace=0.0, wspace=0.)
        idx = 0
        speed = np.linspace(1.0, self.pump.speed_allow, speed_count)
        for s in speed:
            self.pump.set_speed(s)
            q = np.arange(self.pump.flow_min, self.pump.flow_max, flow_step)
            self.power(q, s)
            self.eff(q, s)
            axes[0].plot(q, self.power_supply, color=cmap(idx))
            axes[0].plot(q, self.power_shaft, '--', color=cmap(idx+cmap_count))
            axes[0].plot(q, self.power_hydro, ':', color=cmap(idx+cmap_count*2))
            axes[1].plot(q, self.eff_total, color=cmap(idx))
            axes[1].plot(q, self.eff_motor, '--', color=cmap(idx+cmap_count))
            axes[1].plot(q, self.eff_pump, ':', color=cmap(idx+cmap_count*2))
            idx += 1
        axes[0].legend(['Полная', 'На валу', 'Гидравлическая'], title='Мощность')
        axes[1].legend(['Общий', 'Двигатель', 'Гидравлический'], title='КПД агрегата')
        fig.suptitle('Рабочие характеристики насосного агрегата')
        axes[0].set_ylabel('$P,\, кВт$')
        axes[0].set_ylim(0, None)
        axes[1].set_ylabel('$\eta,\, \%$')
        axes[1].set_ylim(0, None)
        plt.xlabel('$Q,\, м^3/ч$')
        plt.axis([0, flow_xlim, None, None])
        plt.show()

### Набор насосных агрегатов
Предполагаем, что насосы включены параллельно.

Кривая сопротивления системы:

$H(Q) = H_0 + C_S \cdot Q^2$

Задание диапазонов расчета расходу и напору $Q_{max}$ и $H_{max}$ с шагом $Q_{step}$ и $H_{step}$, соответственно

In [1]:
class PumpBattery:

    def __init__(self, flow_step, head_step, files=None, equals=True, speed_min=0.1, speed_max=1.0, speed_num=2501):
        self.units = []
        self.equals = equals
        for file in files:
            self.units.append(PumpUnit(file))
        self.units_num = len(self.units)
        #self.groups = ([1] * (self.units_num)) if groups is None else groups
        #if len(self.groups) != self.units_num:
            #print('Ошибка, несоответствие количества насосов в группах насосов!')
        self.flow_step = flow_step
        self.head_step = head_step
        self.speed_min = speed_min
        self.speed_max = speed_max if speed_max <= 1.0 else 1.0
        self.speed_num = speed_num
        self.create_arrays()
        self.hist_power_supply = pd.Series([])
        self.hist_power_hydro = pd.Series([])
        self.hist_eff_total = pd.Series([])
        self.hist_head = pd.Series([])
        self.hist_flow = pd.Series([])

    def historic(self, kind, file_name, unit_idx=None, convert=False):
        if kind == 'link':
            self.link = LinkSeries(file_name)
        elif unit_idx is None:
            self.kind_historic(kind, file_name, self.link, convert)
        else:
            self.units[unit_idx].kind_historic(kind, file_name, self.link, convert)

    def kind_historic(self, kind, file_name, link, convert):
        if kind == 'head_inflow':
            self.hs_head_inflow = HistoricSeries(file_name, link, convert)
            self.hist_head_inflow = self.hs_head_inflow.adapt
        elif kind == 'head_outflow':
            self.hs_head_outflow = HistoricSeries(file_name, link, convert)
            self.hist_head_outflow = self.hs_head_outflow.adapt
        elif kind == 'flow':
            self.hs_flow = HistoricSeries(file_name, link, convert)
            self.hist_flow = self.hs_flow.adapt
        elif kind == 'power_supply':
            self.hs_power_supply = HistoricSeries(file_name, link, convert)
            self.hist_power_supply = self.hs_power_supply.adapt

    def eff_sum(self, eff_arr):
        sum_eff = 0
        for eff in eff_arr:
            sum_eff += 1 / eff
        return len(eff_arr) * sum_eff**(-1)

    def eff_mutations_array(self, mutas_idx, count, head_idx, flow_sum_idxs):
        eff_mut_arr = []
        for item in range(len(flow_sum_idxs)):
            eff_arr = []
            unit_idx = 0
            for flow_idx in flow_sum_idxs[item]:
                eff_arr.append(self.mutations[mutas_idx][unit_idx].eff_matrix[head_idx][flow_idx])
                unit_idx += 1
            eff_mut_arr.append(self.eff_sum(eff_arr))
        return max(eff_mut_arr)

    def flow_points_max(self):
        flow = 0
        for unit in self.units:
            flow += unit.pump.flow_rated_max
        return flow

    def calc_flow_idx_combi(self, mutas_idx, count, head_idx, flow_idx):
        if count == 1:
            return flow_idx
        flow_idx_max = []
        for unit in self.mutations[mutas_idx][0:count]:
            flow_idx_max.append(unit.flow_idx_max[head_idx])
        fic = max(flow_idx_max)[0]
        return fic if fic < flow_idx else flow_idx

    def calc_flow_sum_idxs(self, count, flow_idx, flow_idx_combi):
        flow_sum_idxs = []
        if self.equals:
            product_start = flow_idx // count
            product_stop = product_start + flow_idx%count + 1
        else:
            product_start = 0
            product_stop = flow_idx_combi + 1
        for flow_idxs in itertools.product(range(product_start, product_stop), repeat=count):
            if sum(flow_idxs) == flow_idx:
                flow_sum_idxs.append(flow_idxs)
        return flow_sum_idxs

    def head_points_max(self):
        head = []
        for unit in self.units:
            head.append(unit.pump.head_rated_max)
        return max(head)

    def calc_head_allow_range(self):
        self.head_min_idx = np.zeros(self.mutas_num*self.units_num, dtype=int)
        self.head_min_idx = self.head_min_idx.reshape((self.mutas_num, self.units_num))
        self.head_max_idx = np.zeros(self.mutas_num*self.units_num, dtype=int)
        self.head_max_idx = self.head_max_idx.reshape((self.mutas_num, self.units_num))
        for mutas_idx in range(self.mutas_num):
            for unit_idx in range(self.units_num):
                count = unit_idx + 1
                head_allow_min = max(self.mutations[mutas_idx][0:count],
                                                                 key=attrgetter('pump.head_allow_min')).pump.head_allow_min
                head_allow_max = min(self.mutations[mutas_idx][0:count], 
                                                                 key=attrgetter('pump.head_allow_max')).pump.head_allow_max
                self.head_min_idx[mutas_idx][unit_idx] = find_index(self.head_points, head_allow_min)
                self.head_max_idx[mutas_idx][unit_idx] = find_index(self.head_points, head_allow_max)

    def calc_flow_idx_max_sum(self, mutas_idx, count, head_idx):
        flow_idx_max_sum = 0
        for unit in self.mutations[mutas_idx][0:count]:
            flow_idx_max_sum += unit.flow_idx_max[head_idx]
        return flow_idx_max_sum[0]

    def append_unit(self, file):
        self.units.append(PumpUnit(file))

    def create_arrays(self):
        self.mutations = list(itertools.permutations(self.units))
        self.mutas_num = len(self.mutations)
        self.flow_points = np.arange(0.0, self.flow_points_max()+self.flow_step, self.flow_step)
        self.head_points = np.arange(0.0, self.head_points_max()+self.head_step, self.head_step)
        self.speed_points = np.linspace(self.speed_min, self.speed_max, self.speed_num)
        self.flow_num = len(self.flow_points)
        self.head_num = len(self.head_points)
        self.eff_total_matrix = np.zeros(self.mutas_num*self.head_num*self.flow_num)
        self.eff_total_matrix = self.eff_total_matrix.reshape((self.mutas_num, self.head_num, self.flow_num))
        self.count_total_matrix = np.zeros(self.mutas_num*self.head_num*self.flow_num)
        self.count_total_matrix = self.count_total_matrix.reshape((self.mutas_num, self.head_num, self.flow_num))
        #self.count_total_matrix = np.full((self.mutas_num, self.head_num, self.flow_num), -1)
        self.calc_head_allow_range()
        self.X, self.Y = np.meshgrid(self.flow_points, self.head_points)

    def calc_units_matrices(self):
        for unit in self.units:
            unit.calc_matrices(self.flow_points, self.flow_num, self.head_points, self.head_num, self.speed_points)

    def calc_mutations_matrices(self, mutas_idx=None):
        if mutas_idx is None:
            for mut_idx in range(self.mutas_num):
                self.calc_mutation_matrix(mut_idx)
        else:
            self.calc_mutation_matrix(mutas_idx)

    def calc_mutation_matrix(self, mutas_idx):
        eff_matrix = np.zeros(self.head_num*self.flow_num*self.units_num)
        eff_matrix = eff_matrix.reshape((self.head_num, self.flow_num, self.units_num))
        for unit_idx in range(self.units_num):
            count = unit_idx + 1
            for head_idx in range(self.head_min_idx[mutas_idx][unit_idx], self.head_max_idx[mutas_idx][unit_idx]+1):
                flow_idx_max_sum = self.calc_flow_idx_max_sum(mutas_idx, count, head_idx)
                for flow_idx in range(flow_idx_max_sum+1):
                    flow_idx_combi = self.calc_flow_idx_combi(mutas_idx, count, head_idx, flow_idx)
                    flow_sum_idxs = self.calc_flow_sum_idxs(count, flow_idx, flow_idx_combi)
                    eff_mut_arr = self.eff_mutations_array(mutas_idx, count, head_idx, flow_sum_idxs)
                    eff_matrix[head_idx][flow_idx][unit_idx] = eff_mut_arr
        for unit_idx in range(self.units_num):
            for head_idx in range(self.head_min_idx[mutas_idx][unit_idx], self.head_max_idx[mutas_idx][unit_idx]+1):
                flow_idx_max_sum = self.calc_flow_idx_max_sum(mutas_idx, count, head_idx)
                for flow_idx in range(flow_idx_max_sum+1):
                    arg_max = np.argmax(eff_matrix[head_idx][flow_idx])
                    self.eff_total_matrix[mutas_idx][head_idx][flow_idx] = eff_matrix[head_idx][flow_idx][arg_max]
                    self.count_total_matrix[mutas_idx][head_idx][flow_idx] = arg_max + 1
                    
    def plot_eff_maps(self):
        levels = np.linspace(10, 80, 15)
        cmap = plt.cm.get_cmap('jet', 15)
        #cmap.set_bad('white',1.)
        fig, axes = plt.subplots(nrows=self.units_num, ncols=1,
                                 sharex=True, sharey=False,
                                 figsize=(7, 5*self.units_num),
                                 constrained_layout=True)
        fig.set_constrained_layout_pads(w_pad=4./72., h_pad=4./72., hspace=0.0, wspace=0.)
        fig.suptitle('Характеристики КПД насосных агрегатов')
        for unit_idx in range(self.units_num):
            Z = self.units[unit_idx].eff_matrix
            #Zm = np.ma.masked_less(Z, 0)
            #data_masked = np.ma.masked_where(data == 0, data)
            #Zm = np.ma.array(Z, mask=np.isnan(Z))
            contours = axes[unit_idx].contourf(self.X, self.Y, Z, levels, cmap=cmap, zorder=0)
            #contours = axes[unit_idx].imshow(Zm, cmap=cmap, interpolation='nearest')
            axes[unit_idx].set_ylabel('$H,\, м$')
        plt.xlabel('$Q,\, м^3/ч$')
        cbar_ax = fig.add_axes([1.05, 0.1/self.units_num, 0.03, 0.8/self.units_num])
        fig.colorbar(contours, ax=axes[0], cax=cbar_ax)
        plt.axis([0, None, 0, None])
        plt.show()

    def plot_eff_total_map(self, mutas_idx):
        levels = np.linspace(10, 80, 15)
        cmap = plt.cm.get_cmap('jet', 15)
        fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 5), constrained_layout=True)
        fig.set_constrained_layout_pads(w_pad=4./72., h_pad=4./72., hspace=0.0, wspace=0.)
        Z = self.eff_total_matrix[mutas_idx]
        contours = axes.contourf(self.X, self.Y, Z, levels, cmap=cmap, zorder=0)
        fig.suptitle(f'Совместная характеристика КПД насосных агрегатов. Вариант {mutas_idx+1}')
        axes.set_xlabel('$Q,\, м^3/ч$')
        axes.set_ylabel('$H,\, м$')
        cbar_ax = fig.add_axes([1.05, 0.1, 0.03, 0.8])
        fig.colorbar(contours, ax=axes, cax=cbar_ax)
        plt.axis([0, None, 0, None])
        plt.show()

    def plot_count_total_map(self, mutas_idx):
        levels = np.linspace(0.5, self.units_num+0.5, self.units_num+1)
        cmap = plt.cm.get_cmap('jet')
        fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 5), constrained_layout=True)
        fig.set_constrained_layout_pads(w_pad=4./72., h_pad=4./72., hspace=0.0, wspace=0.)
        Z = self.count_total_matrix[mutas_idx]
        contours = axes.contourf(self.X, self.Y, Z, levels, cmap=cmap, zorder=0)
        fig.suptitle(f'Оптимальное количество насосных агрегатов. Вариант {mutas_idx+1}')
        axes.set_xlabel('$Q,\, м^3/ч$')
        axes.set_ylabel('$H,\, м$')
        cbar_ax = fig.add_axes([1.05, 0.1, 0.03, 0.8])
        ticks = np.array([x for x in range(1, self.units_num+1)])
        fig.colorbar(contours, ax=axes, cax=cbar_ax, ticks=ticks)
        plt.axis([0, None, 0, None])
        plt.show()

In [5]:
class LinkSeries:

    def __init__(self, file_name, freq='1S'):
        self.raw = pd.read_csv(file_name, sep=';', decimal=',', header=0,
                               parse_dates=[0], index_col=0, squeeze=True, date_parser=dateparser)
        x = [pd.Interval(self.raw.index[i], self.raw.index[i+1]).length for i in range(self.raw.size-1)]
        self.start = self.raw.index[np.argmax(x)]
        self.stop = self.raw.index[np.argmax(x)+1]
        self.full_range = pd.date_range(self.start, self.stop, freq=freq)

In [2]:
class HistoricSeries:

    def __init__(self, file_name, link, convert=False, discrete=False, freq='1S', window=15):
        self.raw = pd.read_csv(file_name, sep=';', decimal=',', header=0,
                               parse_dates=[0], index_col=0, squeeze=True, date_parser=dateparser)
        if convert:
            self.raw = convert_unit('head', self.raw, 'kgf_cm2', 'm')
        self.create_range(link)
        self.create_sampled(freq, discrete)
        self.create_reindex(link)
        self.create_mean(window, discrete)

    def create_range(self, link):
        self.range = self.raw.loc[link.start:link.stop]

    def create_sampled(self, freq, discrete):
        if discrete:
            self.sampled = self.range.resample(freq).interpolate(method='nearest')
        else:
            self.sampled = self.range.resample(freq).mean().interpolate(method='linear')

    def create_reindex(self, link):
        self.reindex = self.sampled.reindex(link.full_range, method="nearest")
    
    def create_mean(self, window, discrete):
        if discrete:
            self.adapt = self.reindex
        else:
            self.adapt = self.reindex.rolling(window, center=True).mean().fillna(0)

In [ ]:
class WorkSeries(HistoricSeries):

    def __init__(self, file_name, link, discrete=True, freq='1S'):
        HistoricSeries.__init__(self, file_name, link, freq, discrete)
        self.yticks = [0, 1]
        self.ylabels = ['Стоп', 'Работа']